In [1]:
import struct
import wave
import os
import time
import tkinter

import pyaudio
import numpy as np
import matplotlib.pyplot as plt

%matplotlib tk

In [2]:
CHUNK = 1024 * 4
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100

In [3]:
def wave_recording():    
    p = pyaudio.PyAudio()

    stream = p.open(
        format=FORMAT,
        channels=CHANNELS,
        rate=RATE,
        input=True,
        output=True,
        frames_per_buffer=CHUNK
    )
    
    fig, ax = plt.subplots(1, figsize=(10, 2))
    x = np.arange(0, CHUNK)
    line, = ax.plot(x, np.random.rand(CHUNK))

    ax.set_xlabel('samples')
    ax.set_ylabel('volume')
    ax.set_ylim(0, 255)
    ax.set_xlim(0,  CHUNK)
    plt.setp(ax, xticks=[0, CHUNK//2, CHUNK], yticks=[0, 128, 255])

    frames = []
    print("Recording ...")
    
    while True:
        data = stream.read(CHUNK) 
        frames.append(data)

        data_int = struct.unpack(str(len(data)) + 'B', data)
        data_np = np.array(data_int, dtype='B')[::2]+ 127

        line.set_ydata(data_np)

        try:
            fig.canvas.draw()
            fig.canvas.flush_events()

        except tkinter.TclError:
            stream.stop_stream()
            stream.close()
            p.terminate()
            break

    print("Done Recording")

    if not os.path.exists('Output'):
        os.mkdir('Output')
    WAVE_OUTPUT_FILENAME = 'Output' + str(time.time_ns())+'.wav'
    print("File : ", WAVE_OUTPUT_FILENAME)

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

In [4]:
wave_recording()

Recording ...
Done Recording
File :  1627733045771035100.wav
